<a href="https://colab.research.google.com/github/vksyr/IST718/blob/master/HW3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd  # data frame operations
import numpy as np  # arrays and math functions
from scipy.stats import uniform  # for training-and-test split
import statsmodels.api as sm  # statistical models (including regression)
import statsmodels.formula.api as smf  # R-like model specification
import matplotlib.pyplot as plt  # 2D plotting
import seaborn as sns  # PROVIDES TRELLIS AND SMALL MULTIPLE PLOTTING
import math

import pandas as pd


In [0]:
# Dataset was converted to CSV with this script: 
# https://pjreddie.com/projects/mnist-in-csv/
def convert(imgf, labelf, outf, n):
    f = open(imgf, "rb")
    o = open(outf, "w")
    l = open(labelf, "rb")

    f.read(16)
    l.read(8)
    images = []

    for i in range(n):
        image = [ord(l.read(1))]
        for j in range(28*28):
            image.append(ord(f.read(1)))
        images.append(image)

    for image in images:
        o.write(",".join(str(pix) for pix in image)+"\n")
    f.close()
    o.close()
    l.close()

convert("/content/drive/My Drive/IST718/HW3/train-images-idx3-ubyte", "/content/drive/My Drive/IST718/HW3/train-labels-idx1-ubyte",
        "/content/drive/My Drive/IST718/HW3/mnist_train.csv", 60000)
convert("/content/drive/My Drive/IST718/HW3/t10k-images-idx3-ubyte", "/content/drive/My Drive/IST718/HW3/t10k-labels-idx1-ubyte",
        "/content/drive/My Drive/IST718/HW3/mnist_test.csv", 10000)

In [0]:
# create pandas df
cols = np.array(['label'])
for i in range(1,785):
  cols = np.append(cols, ['pixel' + str(i)])

train_images = pd.read_csv('/content/drive/My Drive/IST718/HW3/mnist_train.csv', 
                          header=None, names=cols)
test_images = pd.read_csv('/content/drive/My Drive/IST718/HW3/mnist_test.csv', 
                          header=None, names=cols)

In [0]:
'''
# plotting an image
import matplotlib.pyplot as plt
image = np.asarray(train_images.loc[0][1:]).squeeze()
plt.imshow(image)
plt.show()


# plotting an image
import matplotlib.pyplot as plt
image = np.asarray(data[0]).squeeze()
plt.imshow(image)
plt.show()
'''

Tensorflow

In [0]:
import tensorflow as tf

In [0]:
# MODEL
# CREATE PLACEHOLDER VARIABLES FOR OPERATION MANIPULATION
# THE 784 MATCHES THE VECTOR SIZE OF THE MNIST IMAGES - 28*28
x = tf.placeholder(tf.float32, [None, 784])

# MODEL
# CREATE WEIGHTS & BIASES VARIABLES
# IN TF, OUR MODEL PARAMETERS ARE OFTEN MANAGED AS VARIABLES
W = tf.Variable(tf.zeros([784, 10]))
b = tf.Variable(tf.zeros([10]))

# MODEL
# CREATE MODEL - USES SOFTMAX AS THE ACTIVATION FUNCTION
# REMEMBER GOAL FOR ACTIVATION FUNCTION IS TO "SHAPE" THE 
# OUTPUT INTO A PROBABILITY DISTRO OVER THE 10 CLASSES
y = tf.nn.softmax(tf.matmul(x, W) + b)

# MODEL
# CREATE PREDICTED VARIABLE Y-HAT
# AND USE CROSS ENTROPY TO DETERMINE LOSS
# CROSS ENTROPY - HOW INEFFICIENT ARE OUR PREDICTIONS?
y_ = tf.placeholder(tf.float32, [None, 10])
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1]))

# MODEL
# TRAIN USING GRADIENT DESCENT
# LEARNING RATE AT MIDPOINT - .5 - MAKE SMALL ADJUSTMENTS TO MINIMIZE COST
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)



In [0]:
# pandas df to tensorflow dataset
mnist_train = tf.data.Dataset.from_tensor_slices((train_images.iloc[:,1:785]))

In [54]:
titanic_file = tf.keras.utils.get_file("train.csv", "https://storage.googleapis.com/tf-datasets/titanic/train.csv")

32768/30874 [===============================] - 0s 0us/step


In [0]:
df = pd.read_csv(titanic_file, index_col=None)
titanic_slices = tf.data.Dataset.from_tensor_slices(dict(df))

import tensorflow.contrib.eager as tfe
dataset = titanic_slices.batch(5)
for batch in tfe.Iterator(dataset):
     print(batch)

for feature_batch in titanic_slices.take(1):
  for key, value in feature_batch.items():
    print("  {!r:20s}: {}".format(key, value))


In [0]:
for feat, targ in mnist_train.take(5):
  print ('Features: {}, Target: {}'.format(feat, targ))



for feature_batch in mnist_train.take(1):
  for key, value in feature_batch.items():
    print("  {!r:20s}: {}".format(key, value))

In [0]:
# MODEL - RUN
sess = tf.InteractiveSession()
tf.global_variables_initializer().run()

batches = titanic_slices.batch(100)
for batch_xs, batch_ys in tfe.Iterator(batches):
     print(batch_xs)


'''
for _ in range(10000):
  batch_xs, batch_ys = mnist_train.batch(100)
  sess.run(train_step, feed_dict={x: batch_xs, y_: batch_ys})
'''